# Import

In [1]:
import pandas as pd
import os
from utils import *

# Constants & Parameters

In [2]:
PHAVER_PROFILE_ID = os.getenv("PHAVER_PROFILE_ID")
LIMIT_PER_REQUEST = 1000
MAX_FOLLOWINGS_REQUESTED = 5000

# Request Token

In [3]:
access_token = request_access_token()
addToClipBoard("Bearer " + access_token)

# Request Followings

In [ ]:
# Fetch all followings for the given profile ID
followings = get_all_followings(PHAVER_PROFILE_ID, LIMIT_PER_REQUEST, MAX_FOLLOWINGS_REQUESTED, access_token)

# Get the points for each following
# followings = get_all_points(followings, access_token)

# Save the followings to followings.json
save_to_json(followings, 'data/followings.json')

print(f"Successfully saved {len(followings)} followings to 'data/followings.json'.")


# Convert JSON to DataFrame

In [64]:
# Flatten the data
followings_df = flatten_followings(followings)

# Add the current datetime to the DataFrame
followings_df['updatedAt'] = pd.Timestamp.now(tz='UTC')

# Add a tempo column to mark unfollowed users
followings_df['tempo_unfollowed'] = False

# Save the DataFrame to a CSV file
followings_df.to_csv('data/followings.csv', index=False)


# Load Saved Data

In [12]:
followings_df = pd.read_csv('../data/followings.csv')

In [10]:
followings_df[followings_df['tempo_unfollowed']==True]

,id,username,profileCreatedAt,followingDate,followerCount,followingCount,points,credLevel,badge,phaverFrens,verification,verified,isUserFollowing,lensProfile.lensHandle,lensProfile.isUserFollowing,farcasterProfile.name,farcasterProfile.isUserFollowing,updatedAt,tempo_unfollowed


# Unfollow Profiles

In [14]:
# Filter Profiles to Unfollow
profiles_to_unfollow_df = followings_df[
    (followings_df['followerCount'] < 1000) & 
    (followings_df['credLevel'] < 2) &
    (followings_df['badge'].isnull()) &
    (followings_df['verification'].isnull()) &
    # (followings_df['lensProfile.isUserFollowing'] != True) &
    (followings_df['tempo_unfollowed'] == False) 
]
print(f"Found {len(profiles_to_unfollow_df)} profiles to unfollow.")
profiles_to_unfollow_df

Found 768 profiles to unfollow.


,id,username,profileCreatedAt,followingDate,followerCount,followingCount,points,credLevel,badge,phaverFrens,verification,verified,isUserFollowing,lensProfile.lensHandle,lensProfile.isUserFollowing,farcasterProfile.name,farcasterProfile.isUserFollowing,updatedAt,tempo_unfollowed
199,a479ec8d-5da1-4187-b621-af1fc99de44d,a479ec8d-5da1-4187-b621-af1fc99de44d,2024-05-29T12:52:00.621106+00:00,2024-07-10T14:51:27.04669+00:00,48,92,NaN,1,NaN,False,NaN,False,True,jigen.lens,True,NaN,NaN,2024-09-21 22:23:37.739096+00:00,False
200,a3d6d088-dc80-4139-abf3-f60e55f44460,a3d6d088-dc80-4139-abf3-f60e55f44460,2024-03-27T03:50:06.34067+00:00,2024-07-10T14:51:25.437786+00:00,158,549,NaN,1,NaN,False,NaN,False,True,p99199.lens,True,NaN,NaN,2024-09-21 22:23:37.739096+00:00,False
201,24fdd94f-b1e5-41e9-a043-5b9bafc27091,24fdd94f-b1e5-41e9-a043-5b9bafc27091,2024-03-27T03:55:05.804875+00:00,2024-07-10T14:51:24.962908+00:00,85,386,NaN,1,NaN,False,NaN,False,True,m13268.lens,True,NaN,NaN,2024-09-21 22:23:37.739096+00:00,False
202,649ac3d9-5873-4971-93b8-e1b93d03b1f4,649ac3d9-5873-4971-93b8-e1b93d03b1f4,2024-04-22T08:19:07.297627+00:00,2024-07-10T14:51:24.516447+00:00,124,512,NaN,1,NaN,False,NaN,False,True,kzsjtxsj7966.lens,True,NaN,NaN,2024-09-21 22:23:37.739096+00:00,False
203,5a10846a-dc67-4b96-96b8-7a5f3520ce3c,5a10846a-dc67-4b96-96b8-7a5f3520ce3c,2024-04-22T10:39:04.084549+00:00,2024-07-10T14:51:24.101787+00:00,78,469,NaN,1,NaN,False,NaN,False,True,gdqdsjyj.lens,True,NaN,NaN,2024-09-21 22:23:37.739096+00:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570,rgkyWX7eIsVEw5IMxIbKrPsX0hf2,ajushi58,2024-02-15T18:38:15.215362+00:00,2024-04-25T05:18:25.803493+00:00,28,8,NaN,1,NaN,False,NaN,False,True,ajushi.lens,True,ajushi,False,2024-09-21 22:23:37.739096+00:00,False
2583,b4IrLeLmFzYrYR2k3Orn7f51KOA3,linhvu1325,2022-03-18T01:47:29.183689+00:00,2024-04-23T06:03:53.922238+00:00,558,407,NaN,1,NaN,False,NaN,False,True,linhvu1325.lens,True,lineapunk,False,2024-09-21 22:23:37.739096+00:00,False
2584,quMKFL67LJb5wqI3R03NQ4lHzMj2,btcst,2022-07-12T13:34:39.458879+00:00,2024-04-22T20:46:55.440394+00:00,934,718,NaN,1,NaN,False,NaN,False,True,aliceband.lens,True,milad,False,2024-09-21 22:23:37.739096+00:00,False
2590,bVAUH7BRkyX0IecFhNucxVpu3XF2,Mirac.eth,2023-03-26T19:22:57.236635+00:00,2024-04-21T18:37:30.993916+00:00,149,71,NaN,1,NaN,False,NaN,False,True,mirac.lens,True,mirac,False,2024-09-21 22:23:37.739096+00:00,False


In [ ]:
try:
    # Unfollow Profiles
    for index, following in profiles_to_unfollow_df.iterrows():
        unfollowed = unfollow_user(PHAVER_PROFILE_ID, following['id'], 'all', access_token)
        if unfollowed:
            print(f"Unfollowed {following['username']}")
            followings_df.loc[index, 'tempo_unfollowed'] = True
except Exception as e:
    # Save modified following list
    followings_df.to_csv('../data/followings.csv', index=False)
    raise e
